# Assignment 1

1. Реализовать Блум фильтр с помощью битового массива.

Например, вы можете использовать [битовые операции](https://wiki.python.org/moin/BitwiseOperators) или библиотеку `bitarray`.

2. Провести численный эксперимент при false positive rate  $fpr = 0.05$ и количестве объектов $S = 1 000 000$. 

Убедиться, что полученные на семинаре оптимальные параметры фильтра позволяют достичь заявленного false positive rate.
Посчитать  $\frac {\epsilon - \hat \epsilon} \epsilon$, где $\hat \epsilon$ - ваша экспериментальная оценка false positive rate. В качестве объектов используйте строки.

## Задание 1: _фильтр Блума_

In [1]:
import random

In [2]:
class BloomFilter:
    
    """An instance of Bloom Filter.
    
    Params: 
    1) k - length of a filter,
    2) hash_quantity - quantity of hash functions
    
    Methods:
    1) insert(value) - given a variable, performs hashing 
    and makes the necessary changes to a bit array.
    2) lookup(value) - given a variable, checks if it had 
    appeared before.
    """
    
    def __init__(self, k, hash_quantity):
        self.k = k
        self.array = [False] * self.k
        self.hashes = []
        for i in range(hash_quantity):
            # hash function = (a * len(value) + b) mod c
            a = random.randint(1,101)
            b = random.randint(1,101)
            c = random.randint(1, k)
            hash_func = (a, b, c)
            self.hashes.append(hash_func)
    
    def __repr__(self):
        """Print the resulting bit array: 
        1 -- True, 0 -- False.
        :returns: 
        """
        array_str = ""
        for item in self.array:
            if item == True:
                array_str += "1"
            else:
                array_str += "0"
        return array_str
    
    def do_hash(self, value):
        """Performs the hashing of a given value and returns 
        the indices to be changed in the bit array."""
        value = len(value)
        indices = []
        for hash_func in self.hashes:
            a, b, k = hash_func
            res = ((a * value) + b) % self.k
            indices.append(res)
        return indices
    
    def insert(self, value):
        indices = self.do_hash(value)
        for ind in indices:
            self.array[ind] = True
    
    def lookup(self, value):
        indices = self.do_hash(value)
        for ind in indices:
            if self.array[ind] != True:
                return False
        return True

## Задание 2: _эксперимент_

Мы фиксируем $S = 1000000$ (кол-во элементов) и FPR $\epsilon = 0.05$; получается, что 
$$ n = \frac{1}{(\ln 2)^2}\cdot|S|\cdot ln(\frac{1}{\epsilon}) \\
n = \frac{1}{(\ln 2)^2}\cdot 1000000 \cdot ln(\frac{1}{0.05}) \\
n = 6.23522 \cdot 10^6 \approx 6 235 220,$$
т.е. нам нужен фильтр Блума размером 6 235 220.

In [3]:
import string

In [4]:
S = 1000000
test_bf = BloomFilter(k=6235220, hash_quantity=3)

Заполняем фильтр:

In [5]:
def get_lines(S, *previous_set):
    
    lines = set()
    chars = string.ascii_letters + string.digits + string.punctuation
    
    for i in range(S):
        line_len = random.randint(1,101)
        line = "".join(random.choices(chars, k=line_len))
        if not (line in previous_set) or not previous_set:
            lines.add(line)
    return lines

In [6]:
lines = get_lines(S)

In [7]:
for line in lines:
    test_bf.insert(line)

**Проверка на false positives:** сгенерируем до 1 000 000 других строк, которых не было в предыдущем наборе, и проверим, сколько из них "окажутся" в фильтре.

In [8]:
new_lines = get_lines(S, lines)

total = len(new_lines)
f_p = 0

print(total)

986040


In [9]:
for line in new_lines:
    if test_bf.lookup(line) == False:
        print("False positive!")
        f_p += 1
rate = f_p/total * 100
print("False positive rate is: {}".format(rate))

False positive rate is: 0.0
